# 361 A3
Brendan Nisbet <br>
bnis814<br>
332343474

In [1]:
import numpy as np
import operator
from collections import Counter
import math
import csv

### Get top 1000 most common words

I store the top both as just a list of words that I can use as headers later and a dictionary so I can look up specific words.

In [2]:
f = open("trg.csv", "r")

#The frequency of all words
freq = {}

#A list of the top 1000 most common words
best = []

#The frequency top 1000 most common words, beucase I remove them from freq later
total = {}

#Go through each line and 
for line in f:
    #Get each column
    cols = line.split(",")
    #Remove some leftover junk from the csv file
    cols[2] = cols[2].strip("\n")
    cols[2] = cols[2].strip('"')
    
    #Spilt the test column into individual words
    words = cols[2].split(" ")
    
    #For each word add it's frequency to freq
    for word in words:
        if(word in freq):
            freq[word] = freq[word] + 1
        else:
            freq[word] = 1

#Get top 1000 words
for x in range(1000):
    best.append(max(freq.keys(), key=(lambda k: freq[k])))
    total[best[-1]] = freq[best[-1]]
    freq.pop(best[-1])

print("{} ... {}".format(best[:5], best[-5:]))
print(total)

['the', 'of', 'and', 'a', 'in'] ... ['useful', 'symbiont', 'full-orf', 'individual', '29']
{'the': 46507, 'of': 36531, 'and': 24296, 'a': 16548, 'in': 16082, 'to': 12778, 'that': 7743, 'is': 7540, 'genes': 6396, 'with': 6185, 'for': 5830, 'sequence': 5183, 'gene': 5097, 'from': 5081, 'are': 4901, 'was': 4488, 'by': 4317, 'genome': 3728, 'protein': 3493, 'were': 3428, 'as': 3291, 'this': 3136, 'which': 3048, 'an': 3046, 'we': 3030, 'have': 2757, 'amino': 2629, 'two': 2588, 'these': 2549, 'proteins': 2518, 'sequences': 2429, 'acid': 2127, 'human': 2049, 'has': 2048, 'be': 2036, 'dna': 1992, 'been': 1914, 'other': 1843, 'on': 1823, 'at': 1808, 'analysis': 1792, 'cdna': 1773, 'identified': 1645, 'region': 1545, 'or': 1410, 'found': 1384, 'not': 1353, 'chromosome': 1349, 'expression': 1289, 'also': 1250, 'its': 1211, 'between': 1206, 'contains': 1187, 'b': 1187, 'encoding': 1186, 'one': 1165, 'but': 1165, 'cells': 1161, 'three': 1139, 'all': 1139, 'complete': 1129, 'it': 1117, 'sequenced': 

### Process the data to get frequncey tables

In [3]:
#Reimport the csv to make sure nothing has changed.
f = open("trg.csv", "r")

data = []
#Now that the top 1000 words is know, we can go through and make a big array with the frequency of each word for each row
for line in f:
    #Same processing as before
    cols = line.split(",")
    cols[2] = cols[2][1:-2]
    cols[2] = cols[2].strip("\n")
    cols[2] = cols[2].strip('"')
    words = cols[2].split(" ")
    cols = cols[:-1]
    
    #Counter gets the fequency of words in the array
    freqForRow = Counter(words)
    
    if(cols[0] == 'id'):
        #Get a row for headers
        cols += best
    else:
        for word in best:
            if word in words:
                cols.append(int(freqForRow[word]))
            else:
                cols.append(0)
        
        
    data.append(cols)
    

processedData = np.array(data)
print(processedData)

[['id' 'class' 'the' ... 'full-orf' 'individual' '29']
 ['1' 'B' '12' ... '0' '0' '1']
 ['2' 'A' '25' ... '0' '0' '0']
 ...
 ['3998' 'E' '15' ... '0' '0' '0']
 ['3999' 'E' '16' ... '0' '0' '0']
 ['4000' 'B' '10' ... '0' '0' '0']]


### Split data in 5 random groups

In [ ]:
RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

datasplit = [[],[],[],[],[]]

#For every row, randomly put it in one of the groups
for ele in processedData[1:]:#The split data doesn't have headers
    datasplit[np.random.randint(0, 5)].append(ele)
        
datasplit = np.array(datasplit)

for ele in datasplit:
    print(len(ele))

### Simple Naive Bayes 

I started with making the simple Naive Bayes algorithm as described in the lecture. I needed to log the probability before summing them because otherwise, it would underflow. This implementation also has cross-validation to check accuracy.

In [ ]:
classes = ['A', 'B', 'E', 'V'] #0:A, 1:B, 2:E, 3:V
score = np.zeros(len(datasplit))

#For all five validation groups
for w in range(len(datasplit)):
    #classnum is the amount of classes
    classnum = [0,0,0,0]
    #classtotal is the total amount of words in a class
    classtotal = [0,0,0,0]
    #byclass is so I can look up how many times a word appears in a class
    byClass = dict((el,[0,0,0,0]) for el in best)
    
    test = []
    train = []
    #Creat train and test using split data from before
    for z in range(len(datasplit)):
        if(z == w):
            test = datasplit[z].copy()
        else:
            train += datasplit[z].copy()
            
    train = np.array(train)
    test = np.array(test)
    
    for ele in train:
        eleClass = classes.index(ele[1])
        classnum[eleClass] += 1
        for i in range(2, len(ele)):
            classtotal[eleClass] += int(ele[i])
            byClass[data[0][i]][eleClass] += int(ele[i])
            
    correct = 0

    #for each row in test
    for ele in test:
        #classProbs stores the probability that a row is a class
        classProbs = np.zeros(4)
        
        #For the number of classes this row could be (4)
        for i in range(4):
            #Initialize the probablity to be the log of the amount this class occurs over all classes occurring
            classProbs[i] = math.log(classnum[0] / len(train))
            
            #For every word in the top 1000
            for j in range(2, len(ele)):
                #If this word appears in the row
                if float(ele[j]) > 0 :
                    #The count of this word in this class smoothed with 1
                    countWC = (int(byClass[data[0][j]][i]) + 1)
                    #The count of this class smoothed with the number of words (1000)
                    countCV = (int(classtotal[i]) + len(best))
                    
                    #The probility of this word occuring in this class, to the power of the number of times it occurs in this row
                    probofword = math.pow((countWC/countCV), float(ele[j]))
                    
                    #Incease the total probability it's this class (added because everything is a log)
                    classProbs[i] = classProbs[i] + math.log(probofword)
        
        #Get the best guess and check if it's right.
        if(classes[np.argmax(classProbs)] == ele[1]):
            correct += 1
    
    score[w] = (correct/len(test))
    print("Score of fold {}: {:.4f}".format(w+1, score[w]))

print("\n")
print("Mean Score: {:.4f}".format(score.mean()))
print("Standard divaiation: {:.4f}".format(score.std()))

This did alright. It gets about 89% during the cross-validation which is much better than random but not the best.

### First modification

My first modification to the algorithm I was trying to account for the fact that classes in the training data set are distributed very unevenly. To do this I implemented the modifications outlined in section 3.1 of Tackling the Poor Assumptions of Naive Bayes Text Classifiers. 

In [ ]:
classes = ['A', 'B', 'E', 'V'] #0:A, 1:B, 2:E, 3:V
score = np.zeros(len(datasplit))

#For all five validation groups
for w in range(len(datasplit)):
    #classnum is the amount of classes
    classnum = [0,0,0,0]
    #classtotal is the total amount of words in a class
    classtotal = [0,0,0,0]
    #byclass is so I can look up how many times a word appears in a class
    byClass = dict((el,[0,0,0,0]) for el in best)
    
    test = []
    train = []
    #Creat train and test using split data from before
    for z in range(len(datasplit)):
        if(z == w):
            test = datasplit[z].copy()
        else:
            train += datasplit[z].copy()
            
    train = np.array(train)
    test = np.array(test)
    
    for ele in train:
        eleClass = classes.index(ele[1])
        classnum[eleClass] += 1
        for i in range(2, len(ele)):
            classtotal[eleClass] += int(ele[i])
            byClass[data[0][i]][eleClass] += int(ele[i])
            

    correct = 0

    #for each row in test
    for ele in test:
        #classProbs stores the probability that a row is a class
        classProbs = np.zeros(4)
        
        #For the number of classes this row could be (4)
        for i in range(4):
            #Initialize the probablity to be the log of the amount this class occurs over all classes occurring. 
            classProbs[i] = math.log(classnum[0] / len(train))
            
            #For every word in the top 1000
            for j in range(2, len(ele)):
                #If this word appears in the row
                if float(ele[j]) > 0 :
                    #otherWC is the number of times a word occurs in a different class to the current
                    otherWC = 1
                    #otherCV is the total number of occurrences of different class to the current
                    otherCV = len(best)
                    
                    #If its the current class, update countWC and countCV, if it's not update otherWC and otherCV
                    for x in range(4):
                        if(x == i):
                            countWC = (int(byClass[data[0][j]][i]) + 1)
                            countCV = (int(classtotal[i]) + len(best))
                        else:
                            otherWC += byClass[data[0][j]][x]
                            otherCV += classtotal[x]
                            
                    #This time the probility it's in this class is over the probabilty it's in a different class
                    probofword = math.pow(((countWC/countCV)/(otherWC/otherCV)), float(ele[j]))
                    #Incease the total probability it's this class (added because everything is a log)
                    classProbs[i] = classProbs[i] + math.log(probofword)
                    
        #Get the best guess and check if it's right.
        if(classes[np.argmax(classProbs)] == ele[1]):
            correct += 1
            
    score[w] = (correct/len(test))
    print("Score of fold {}: {:.4f}".format(w+1, score[w]))

print("\n")
print("Mean Score: {:.4f}".format(score.mean()))
print("Standard divaiation: {:.4f}".format(score.std()))

This worked quite well with an increase in accuracy of about 2 percentage points. 

### Second modification

For the second extension, I tried to account for words like "the" that appear many times in all rows. I did this by implementing section 4.2 of Tackling the Poor Assumptions of Naive Bayes Text Classifiers.

In [ ]:
classes = ['A', 'B', 'E', 'V'] #0:A, 1:B, 2:E, 3:V
score = np.zeros(len(datasplit))

#For all five validation groups
for w in range(len(datasplit)):
    #classnum is the amount of classes
    classnum = [0,0,0,0]
    #classtotal is the total amount of words in a class
    classtotal = [0,0,0,0]
    #byclass is so I can look up how many times a word appears in a class
    byClass = dict((el,[0,0,0,0]) for el in best)
    #Occurs is the number of rows a words appares in
    occurs = np.zeros(1000)
    
    test = []
    train = []
    #Creat train and test using split data from before
    for z in range(len(datasplit)):
        if(z == w):
            test = datasplit[z].copy()
        else:
            train += datasplit[z].copy()
            
    train = np.array(train)
    test = np.array(test)
    
    for ele in train:
        eleClass = classes.index(ele[1])
        classnum[eleClass] += 1
        for i in range(2, len(ele)):
            classtotal[eleClass] += int(ele[i])
            byClass[data[0][i]][eleClass] += int(ele[i])
            if(int(ele[i]) > 0):
                occurs[i-2] += 1
            
    correct = 0

    #for each row in test
    for ele in test:
        #classProbs stores the probability that a row is a class
        classProbs = np.zeros(4)
        
        #For the number of classes this row could be (4)
        for i in range(4):
            #Initialize the probablity to be the log of the amount this class occurs over all classes occurring. 
            classProbs[i] = math.log(classnum[0] / len(train))
            
            #For every word in the top 1000
            for j in range(2, len(ele)):
                #If this word appears in the row
                if float(ele[j]) > 0 :
                    #otherWC is the number of times a word occurs in a different class to the current
                    otherWC = 1
                    #otherCV is the total number of occurrences of different class to the current
                    otherCV = len(best)
                    
                    #If its the current class, update countWC and countCV, if it's not update otherWC and otherCV
                    for x in range(4):
                        if(x == i):
                            #Transforming by document frequency of this word
                            countWC = (int(byClass[data[0][j]][i]) + 1) * math.log(len(train)/occurs[j-2])
                            countCV = (int(classtotal[i]) + len(best))
                        else:
                            #Transforming by document frequency of this word
                            otherWC += byClass[data[0][j]][x] * math.log(len(train)/occurs[j-2])
                            otherCV += classtotal[x]

                    #Same as before
                    probs = math.pow(((countWC/countCV)/(otherWC/otherCV)), float(ele[j]))
                    classProbs[i] = classProbs[i] + math.log(probs)
                    
                    
        #Get the best guess and check if it's right.
        if(classes[np.argmax(classProbs)] == ele[1]):
            correct += 1
            
    score[w] = (correct/len(test))
    print("Score of fold {}: {:.4f}".format(w+1, score[w]))

print("\n")
print("Mean Score: {:.4f}".format(score.mean()))
print("Standard divaiation: {:.4f}".format(score.std()))

The effect of this change is very small with how I've implemented it at less than 1 percentage points which at how small it is could be due to randomness.

## Kaggle

Below is the code I used to get an output I could use for kaggle. I use the whole training set instead of just 4/5ths of it so the score on kaggle is slightly higher than that of any of my validations. It's using the algorithm with both modifications. 

In [4]:
f = open("tst.csv", "r")
data = []
for line in f:
    cols = line.split(",")
    cols[1] = cols[1][1:-2]
    words = cols[1].split(" ")
    cols = cols[:-1]
    freq = Counter(words)
    if(cols[0] == 'id'):
        cols += best
    else:
        for word in best:
            if word in words:
                cols.append(freq[word])
            else:
                cols.append(0)
        
        
    data.append(cols)
    

testData = np.array(data)
print(testData)

[['id' 'the' 'of' ... 'full-orf' 'individual' '29']
 ['1' '10' '9' ... '0' '0' '0']
 ['2' '6' '7' ... '0' '0' '0']
 ...
 ['998' '8' '10' ... '0' '0' '0']
 ['999' '7' '6' ... '0' '0' '0']
 ['1000' '24' '16' ... '0' '0' '0']]


In [7]:
classes = ['A', 'B', 'E', 'V'] #0:A, 1:B, 2:E, 3:V

#classnum is the amount of classes
classnum = [0,0,0,0]
#classtotal is the total amount of words in a class
classtotal = [0,0,0,0]
#byclass is so I can look up how many times a word appears in a class
byClass = dict((el,[0,0,0,0]) for el in best)
#Occurs is the number of rows a words appares in
occurs = np.zeros(1000)

for ele in processedData[1:]:
    eleClass = classes.index(ele[1])
    classnum[eleClass] += 1
    for i in range(2, len(ele)):
        classtotal[eleClass] += int(ele[i])
        byClass[processedData[0][i]][eleClass] += int(ele[i])
        if(int(ele[i]) > 0):
            occurs[i-2] += 1

In [9]:
classes = ['A', 'B', 'E', 'V']

predictions = []

for ele in testData[1:]:
    cat = np.zeros(4)
    for i in range(4):
            cat[i] = math.log(classnum[0] / len(processedData))
    for i in range(4):
        for j in range(2, len(ele)):
            if float(ele[j]) > 0:
                otherWC = 1
                otherCV = len(best)
                
                for x in range(4):
                    if(x == i):
                        countWC = (int(byClass[data[0][j]][i]) + 1) * math.log(len(processedData)/occurs[j-2])
                        countCV = (int(classtotal[i]) + len(best))
                    else:
                        otherWC += byClass[data[0][j]][x] * math.log(len(processedData)/occurs[j-2])
                        otherCV += classtotal[x]

            
                probs = math.pow(((countWC/countCV)/(otherWC/otherCV)), float(ele[j]))
                cat[i] = cat[i] + math.log(probs)
        
    predictions.append(classes[np.argmax(cat)])
               


In [11]:
with open('predictions.csv', mode='w', newline='') as csv_file:
    headers = ['id', 'class']
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writerow({'id':'id', 'class':'class'})
    for i in range(len(predictions)):
        writer.writerow({'id':i+1, 'class':predictions[i]})